In [1]:
from sktime.datasets import load_airline

In [2]:
airline = load_airline()

In [3]:
airline[:10]

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
1949-06    135.0
1949-07    148.0
1949-08    148.0
1949-09    136.0
1949-10    119.0
Freq: M, Name: Number of airline passengers, dtype: float64

## We use Reduction approach in order to convert the timeseries data into (x,y)

In [4]:
features = []
targets = []

for i in range(len(airline)-10):
    features.append(airline[i:i+10])
    targets.append(airline[i+10])

In [5]:
import numpy as np

In [6]:
features = np.array(features)
targets = np.array(targets)

In [7]:
features.shape

(134, 10)

In [8]:
targets.shape

(134,)

In [9]:
train_len = int(len(targets)*0.80)

In [10]:
print(train_len)

107


In [11]:
train_features = features[:train_len]
train_target = targets[:train_len]
test_features = features[train_len:]
test_target = targets[train_len:]

In [12]:
print(train_features.shape)
print(train_target.shape)

(107, 10)
(107,)


In [13]:
print(test_features.shape)
print(test_target.shape)

(27, 10)
(27,)


In [14]:
from sklearn.tree import DecisionTreeRegressor

In [15]:
reg = DecisionTreeRegressor()

In [16]:
reg.fit(train_features, train_target)

DecisionTreeRegressor()

In [17]:
pred_target = reg.predict(test_features)

In [18]:
pred_target

array([491., 422., 422., 422., 348., 348., 465., 505., 505., 491., 505.,
       467., 505., 505., 422., 505., 505., 505., 505., 505., 505., 505.,
       505., 505., 505., 505., 505.])

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_squared_error(test_target, pred_target)

6314.259259259259

### Alternative approach: Using sktime library we can do all above steps easily

In [22]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import GradientBoostingRegressor

In [23]:
y_train, y_test = temporal_train_test_split(airline, test_size=36)

In [24]:
regressor = GradientBoostingRegressor()

In [25]:
forecaster = make_reduction(regressor, window_length=15, strategy="recursive")

In [26]:
forecaster.fit(y_train)

RecursiveTabularRegressionForecaster(estimator=GradientBoostingRegressor(),
                                     window_length=15)

In [27]:
y_pred = forecaster.predict(fh=np.arange(1, len(y_test)+1))

In [30]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [31]:
mean_absolute_percentage_error(y_test, y_pred, symmetric=True)

0.11593642659387288

In [32]:
from sktime.regression.kernel_based import RocketRegressor

In [33]:
reg = RocketRegressor(num_kernels=500)

In [34]:
forecaster2 = make_reduction(reg, window_length=15, strategy="recursive")

In [35]:
forecaster2.fit(y_train)

RecursiveTimeSeriesRegressionForecaster(estimator=RocketRegressor(num_kernels=500),
                                        window_length=15)

In [37]:
y_pred2 = forecaster2.predict(fh=np.arange(1, len(y_test)+1))

In [38]:
mean_absolute_percentage_error(y_test, y_pred2, symmetric=True)

0.3428058684172092